In [3]:
import pybullet as p
import pybullet_data
import time
import random
import os
import numpy as np
import matplotlib.pyplot as plt

# =================================================================
# 1. ENVIRONMENT SETUP (Binder vs. Local)
# =================================================================
def setup_physics_client():
    if 'DISPLAY' not in os.environ or not os.environ['DISPLAY']:
        print("Detected Headless Environment (Binder/Docker Server)")
        # Start a virtual display so the renderer has a 'screen' to draw on
        from pyvirtualdisplay import Display
        display = Display(visible=0, size=(1024, 768))
        display.start()
        client = p.connect(p.DIRECT)
    else:
        print("Detected GUI Environment (Local Ubuntu)")
        client = p.connect(p.GUI)
    
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.setGravity(0, 0, -9.81)
    return client

# =================================================================
# 2. VISUALIZATION HELPER (For Binder Users)
# =================================================================
def show_scene():
    """Renders the current simulation state for the Jupyter Notebook."""
    
    # 1. Manually define the camera position
    # This replaces the failing computeViewMatrixFromVisualizerConfig()
    view_matrix = p.computeViewMatrix(
        cameraEyePosition=[2, 2, 2],    # Where the camera is located (x, y, z)
        cameraTargetPosition=[0, 0, 0], # Where the camera is looking
        cameraUpVector=[0, 0, 1]        # Which way is 'up' (usually Z)
    )
    
    # 2. Set the projection (Perspective)
    proj_matrix = p.computeProjectionMatrixFOV(
        fov=60, aspect=1.0, nearVal=0.1, farVal=100.0
    )
    
    # 3. Capture image
    (_, _, px, _, _) = p.getCameraImage(
        width=640, height=480, 
        viewMatrix=view_matrix, 
        projectionMatrix=proj_matrix,
        renderer=p.ER_TINY_RENDERER
    )
    
    # 4. Process and display
    rgb_array = np.reshape(np.array(px), (480, 640, 4))[:, :, :3]
    plt.figure(figsize=(10, 7))
    plt.imshow(rgb_array)
    plt.axis('off')
    plt.show()

# =================================================================
# 3. ROBOT WORLD LOGIC
# =================================================================
def spawn_lying_cylinder(color):
    """Spawns a small cylinder horizontal on the floor."""
    x, y = random.uniform(-1.5, 1.5), random.uniform(-1.5, 1.5)
    
    # Rotation: 90 degrees around Y axis to make it 'lying'
    orientation = p.getQuaternionFromEuler([0, 1.57, 0])
    
    vis_id = p.createVisualShape(shapeType=p.GEOM_CYLINDER, radius=0.03, length=0.1, rgbaColor=color)
    col_id = p.createCollisionShape(shapeType=p.GEOM_CYLINDER, radius=0.03, height=0.1)
    
    return p.createMultiBody(baseMass=0.5,
                             baseCollisionShapeIndex=col_id,
                             baseVisualShapeIndex=vis_id,
                             basePosition=[x, y, 0.03],
                             baseOrientation=orientation)

def main():
    # Initialize
    client = setup_physics_client()
    
    # Load Floor
    floor_id = p.loadURDF("plane.urdf")
    p.changeVisualShape(floor_id, -1, rgbaColor=[0.2, 0.2, 0.2, 1]) # Dark Gray

    # Spawn Target (Red Lying Cylinder)
    target_red = spawn_lying_cylinder([1, 0, 0, 1])

    # Spawn Obstacle (Blue Cube)
    box_col = p.createCollisionShape(p.GEOM_BOX, halfExtents=[0.15, 0.15, 0.15])
    box_vis = p.createVisualShape(p.GEOM_BOX, halfExtents=[0.15, 0.15, 0.15], rgbaColor=[0, 0, 1, 1])
    obstacle_blue = p.createMultiBody(baseMass=10,
                                      baseCollisionShapeIndex=box_col,
                                      baseVisualShapeIndex=box_vis,
                                      basePosition=[random.uniform(-1, 1), random.uniform(-1, 1), 0.15])

    # Let objects settle
    for _ in range(100):
        p.stepSimulation()

    # Log Coordinates
    pos, _ = p.getBasePositionAndOrientation(target_red)
    print(f"Target Red (Lying) at: {pos}")

    # Set a nice camera angle before rendering
    p.resetDebugVisualizerCamera(cameraDistance=3, cameraYaw=45, cameraPitch=-30, cameraTargetPosition=[0, 0, 0])
    
    # Final Visualization
    show_scene()

    # Clean up
    # p.disconnect() # Uncomment this if running as a script. 
    # Keep commented if you want to inspect variables in Jupyter after the run.

if __name__ == "__main__":
    main()

SyntaxError: invalid syntax (<ipython-input-3-47045b88d941>, line 106)